In [355]:
import sys
import time
import subprocess
import signal
import os
import math
import rospy
import rosbag
import roslib
roslib.load_manifest("robot_comm")
from robot_comm.srv import *
roslib.load_manifest("Mocap")
from Mocap.msg import *
from Mocap.srv import *
import numpy as np

In [356]:
get_cart = rospy.ServiceProxy('/robot_GetCartesian', robot_GetCartesian)
set_work_object = rospy.ServiceProxy('/robot_SetWorkObject', robot_SetWorkObject)
set_tool = rospy.ServiceProxy('/robot_SetTool', robot_SetTool)
set_cart = rospy.ServiceProxy('/robot_SetCartesian', robot_SetCartesian)
#Set Speed
set_speed = rospy.ServiceProxy('/robot_SetSpeed', robot_SetSpeed)
set_speed(10, 5)
get_cart()

x: 611.62
y: 435.75
z: 298.72
q0: 0.0004
qx: -0.7058
qy: -0.7084
qz: 0.0004
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [357]:
set_mocap_tf = rospy.ServiceProxy('/mocap_SetMocapTransformation', mocap_SetMocapTransformation)
#set_mocap_tf(666.06, -256.24, 55.61, 0.4922, 0.49687, 0.50185, 0.50893)
#set_mocap_tf(666.16, -256.27, 55.682, 0.49236, 0.49671, 0.50187, 0.50891)
set_mocap_tf(668.54, -260.52, 56.021, 0.49366, 0.49611, 0.50222, 0.50788)
get_mocap_frame = rospy.ServiceProxy('/mocap_GetMocapFrame', mocap_GetMocapFrame)

In [361]:
get_mocap_frame()

mf: 
  header: 
    seq: 0
    stamp: 
      secs: 1435534323
      nsecs: 723603061
    frame_id: ''
  number: 0
  body_poses: 
    - 
      position: 
        x: 528.757469199
        y: -6.80080365991
        z: 45.2287401263
      orientation: 
        x: 0.300642320697
        y: 0.635914438094
        z: 0.637346032424
        w: 0.314656298086
  uid_markers: 
    markers: 
      - 
        x: 508.782899253
        y: 20.262938234
        z: 46.3141539341
      - 
        x: 513.558035291
        y: -19.1137189706
        z: 46.033033762
      - 
        x: 534.19698005
        y: -10.6241021869
        z: 45.3271757493
      - 
        x: 558.492563078
        y: -17.3683924654
        z: 46.8091925996
  id_marker_sets: 
    - 
      markers: 
        - 
          x: 508.751215807
          y: 20.3087536706
          z: 46.3157019833
        - 
          x: 513.580022143
          y: -19.1265782548
          z: 46.0344851867
        - 
          x: 534.570077159
          y: -10

In [359]:
set_work_object(0,0,0,1,0,0,0)
set_tool(-125,0,115, 0, 0, 1, 0)
get_cart()

x: 500.0
y: 0.0
z: 290.0
q0: 1.0
qx: 0.0
qy: 0.0
qz: 0.0
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [375]:
# Set to pre-push position.
set_speed(10,5)
set_cart(500, 0, 25, 1, 0, 0, 0)
set_speed(5,5)

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [372]:
get_cart()

x: 500.0
y: 0.0
z: 25.0
q0: 1.0
qx: 0.0
qy: 0.0
qz: 0.0
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [321]:
# Set push end position. +x for 50 mm. 
# set_cart(550,0,15,1,0,0,0)
# rosbag command
# rosbag record -O bags/robot_push_many.bag /Mocap /netft_data /robot_CartesianLog
#np.random.uniform(0,1,(3,1))
cur_robot_pos

array([ 499.99,    0.  ,   15.  ])

In [376]:
np.random.seed(10)
dir = np.array([[1, 0, 0],[0, -1, 0],[0, 1, 0]]).transpose()
num_pushes = 10
dist_push = 30 # mm
for i in range(0,num_pushes):
    #c = np.random.rand(3,1)
    c = np.random.uniform(0,1,(3,1))
    c = c / c.sum()
    rand_disp = dir.dot(np.random.rand(3,1))
    #print rand_disp
    r_cart = get_cart()
    cur_robot_pos = np.array([r_cart.x, r_cart.y, r_cart.z])
    nxt_robot_pos = cur_robot_pos + dist_push * rand_disp.transpose()
    #Sample a random angle between -theta to theta.
    angle = np.random.uniform(-np.pi/12,np.pi/12)
    if (nxt_robot_pos[0,2] > 24.99  and cur_robot_pos[2] > 24.99): 
        set_cart(cur_robot_pos[0], cur_robot_pos[1], cur_robot_pos[2], np.cos(angle), 0, 0, np.sin(angle))
        time.sleep(0.75)
        set_cart(nxt_robot_pos[0,0], nxt_robot_pos[0,1], nxt_robot_pos[0,2], np.cos(angle), 0, 0, np.sin(angle))
    #print get_cart()
    time.sleep(0.75)
    #back up a little bit. 
    r_cart = get_cart()
    cur_robot_pos = np.array([r_cart.x, r_cart.y, r_cart.z])
    nxt_robot_pos = cur_robot_pos - 2 * rand_disp.transpose()
    #if (nxt_robot_pos[0,2] > 14.99):
    #    set_cart(nxt_robot_pos[0,0], nxt_robot_pos[0,1], nxt_robot_pos[0,2], np.cos(angle), 0, 0, np.sin(angle))
        #set_cart(nxt_robot_pos[0,0], nxt_robot_pos[0,1], nxt_robot_pos[0,2], 1, 0, 0, 0)

In [304]:
np.random.uniform(-np.pi/4,np.pi/4)
num_pushes

10

In [116]:
os.getcwd()

'/home/jiaji/MLab_EXP/Mocap/PythonExp'

In [377]:
f1 = open("exp4/pos3.txt", "w");
f2 = open("exp4/force3.txt", "w");
f3 = open("exp4/robot3.txt", "w");
bag = rosbag.Bag('bags/robot_multi_push_3out_3.bag')
last_topic = '/netft_data'
ct = 0
last_pos_x = 0
last_pos_y = 0
pose = geometry_msgs.msg.Pose()
force = geometry_msgs.msg.Wrench.force
for topic, msg, t in bag.read_messages(topics=['/Mocap', '/netft_data', '/robot_CartesianLog']):
    s = str(t.to_sec()) + '\n'
    if topic == '/Mocap':
        pos = msg.body_poses[0].position
        ori = msg.body_poses[0].orientation    
        s = s + str(pos.x) + ' ' + str(pos.y) + ' ' + str(pos.z) + ' '
        #Use q0/qw,qx,qy,qz convention.
        s = s + str(ori.w) + ' ' + str(ori.x) + ' ' + str(ori.y) + ' ' + str(ori.z)
        s = s + '\n'
        f1.write(s);  
    if topic == '/netft_data':
        force = msg.wrench.force
        s = s + str(force.x) + ' ' + str(force.y) + ' ' + str(force.z)
        s = s + '\n'
        f2.write(s)
    if topic == '/robot_CartesianLog':
        robot_cart = msg
        s = s + str(robot_cart.x) + ' ' + str(robot_cart.y) + ' ' + str(robot_cart.z) + ' '
        s = s + str(robot_cart.q0) + ' ' + str(robot_cart.qx) + ' ' + str(robot_cart.qy) + ' ' + str(robot_cart.qz)
        s = s + '\n'
        f3.write(s)
bag.close()
f1.close()
f2.close()
f3.close()

In [278]:
set_cart(575,0,290,np.sqrt(2)/2,0,0,np.sqrt(2)/2)  # Point to the left. +y

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [281]:
set_cart(575,0,290,np.sqrt(2)/2,0,0,-np.sqrt(2)/2) # Point to the right. -y


ret: 1
msg: ROBOT_CONTROLLER: OK.

In [354]:
set_cart(500,0,290,1,0,0,0)  # Point to the left. +y

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [11]:
# Just dump the raw mocap body data.
f1 = open("mocap_pos_5.txt", "w")
bag = rosbag.Bag('test_human_push_5.bag')
for topic, msg, t in bag.read_messages(topics=['/Mocap']):
    if topic == '/Mocap':
        pos = msg.body_poses[0].position
        ori = msg.body_poses[0].orientation
        s = str(t.to_sec()) + '\n'
        s = s + str(pos.x) + ' ' + str(pos.y) + ' ' + str(pos.z) + ' '
        s = s + str(ori.w) + ' ' + str(ori.x) + ' ' + str(ori.y) + ' ' + str(ori.z)
        s = s + '\n'
        f1.write(s);
f1.close()

In [86]:
t.to_sec()

1430178331.7931406

In [122]:
set_tool(-100, 0, 110, 0, 0, 1, 0);


In [121]:
set_cart(550, -100, 375, 1, 0, 0, 0)

x: 549.99
y: -99.97
z: 375.0
q0: 0.0
qx: -0.7071
qy: 0.7071
qz: 0.0001
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [123]:
get_joints = rospy.ServiceProxy('/robot_GetJoints', robot_GetJoints)

In [124]:
get_joints()

j1: 0.0
j2: 20.55
j3: 0.92
j4: 0.01
j5: 68.54
j6: 90.0
ret: 1
msg: ROBOT_CONTROLLER: OK.